# Milestone 2
- merge each one's dataset
- descriptive statistics
- visualization (put all vis into another notebook)
- pipeline
- regression


In [1]:
import pandas as pd 

In [8]:
df_agri = pd.read_csv('../data/processed/Agriculture_data.csv')
df_mobile = pd.read_csv('../data/processed/IMF_mobilemoney.csv')
df_con = pd.read_csv('../data/processed/index_merged_asean_sa_data.csv')
df_gfi = pd.read_csv('../data/processed/merged_output.csv')
df_staple = pd.read_csv('../data/processed/StapleFoodStability.csv')


In [50]:
# rename to 'code'
df_agri = df_agri.rename(columns={'countryiso3code': 'code'})
df_con = df_con.rename(columns={'countryiso3code': 'code'})
df_gfi = df_gfi.rename(columns={'countryiso3code': 'code'})

In [38]:
# rename 'COUNTRY ' to 'country'
df_mobile = df_mobile.rename(columns={'COUNTRY': 'country'})
df_staple = df_staple.rename(columns={'Area': 'country'})

In [42]:
# rename 'date' or 'Year' to 'year'
df_con = df_main.rename(columns={'date': 'year'})
df_gfi= df_main.rename(columns={'date': 'year'})
df_agri= df_main.rename(columns={'Year': 'year'})

In [51]:
#df_mobile.head(10)
df_agri.head(10)

,country,year,indicator,code,value,unit,obs_status,decimal,Region
0,Brunei Darussalam,1970,"Population, total",BRN,NaN,NaN,NaN,0,ASEAN
1,Brunei Darussalam,1970,"Population, total",BRN,458949.0,NaN,NaN,0,ASEAN
2,Brunei Darussalam,1970,"Population, total",BRN,455370.0,NaN,NaN,0,ASEAN
3,Brunei Darussalam,1970,"Population, total",BRN,451721.0,NaN,NaN,0,ASEAN
4,Brunei Darussalam,1970,"Population, total",BRN,447404.0,NaN,NaN,0,ASEAN
5,Brunei Darussalam,1970,"Population, total",BRN,442680.0,NaN,NaN,0,ASEAN
6,Brunei Darussalam,1970,"Population, total",BRN,437810.0,NaN,NaN,0,ASEAN
7,Brunei Darussalam,1970,"Population, total",BRN,432772.0,NaN,NaN,0,ASEAN
8,Brunei Darussalam,1970,"Population, total",BRN,427564.0,NaN,NaN,0,ASEAN
9,Brunei Darussalam,1970,"Population, total",BRN,422212.0,NaN,NaN,0,ASEAN


In [52]:
df_gfi.head(5)

,country,year,indicator,code,value,unit,obs_status,decimal,Region
0,Brunei Darussalam,1970,"Population, total",BRN,NaN,NaN,NaN,0,ASEAN
1,Brunei Darussalam,1970,"Population, total",BRN,458949.0,NaN,NaN,0,ASEAN
2,Brunei Darussalam,1970,"Population, total",BRN,455370.0,NaN,NaN,0,ASEAN
3,Brunei Darussalam,1970,"Population, total",BRN,451721.0,NaN,NaN,0,ASEAN
4,Brunei Darussalam,1970,"Population, total",BRN,447404.0,NaN,NaN,0,ASEAN


In [53]:
df_con.head(5)

,country,year,indicator,code,value,unit,obs_status,decimal,Region
0,Brunei Darussalam,1970,"Population, total",BRN,NaN,NaN,NaN,0,ASEAN
1,Brunei Darussalam,1970,"Population, total",BRN,458949.0,NaN,NaN,0,ASEAN
2,Brunei Darussalam,1970,"Population, total",BRN,455370.0,NaN,NaN,0,ASEAN
3,Brunei Darussalam,1970,"Population, total",BRN,451721.0,NaN,NaN,0,ASEAN
4,Brunei Darussalam,1970,"Population, total",BRN,447404.0,NaN,NaN,0,ASEAN


In [54]:
df_staple.head(5)

,Unnamed: 0,country,Year,Food supply quantity (kg/capita/yr)_Cereals - Excluding Beer,Food supply quantity (kg/capita/yr)_Starchy Roots,rolling_std
0,0,Afghanistan,2010,202.73,6.69,NaN
1,1,Afghanistan,2011,197.29,5.72,NaN
2,2,Afghanistan,2012,190.31,6.40,NaN
3,3,Afghanistan,2013,193.09,14.55,NaN
4,4,Afghanistan,2014,190.09,13.46,5.345682


In [55]:
df_agri.head(5)

,country,year,indicator,code,value,unit,obs_status,decimal,Region
0,Brunei Darussalam,1970,"Population, total",BRN,NaN,NaN,NaN,0,ASEAN
1,Brunei Darussalam,1970,"Population, total",BRN,458949.0,NaN,NaN,0,ASEAN
2,Brunei Darussalam,1970,"Population, total",BRN,455370.0,NaN,NaN,0,ASEAN
3,Brunei Darussalam,1970,"Population, total",BRN,451721.0,NaN,NaN,0,ASEAN
4,Brunei Darussalam,1970,"Population, total",BRN,447404.0,NaN,NaN,0,ASEAN


In [65]:
df_con.head(5)

,country,year,indicator,code,value,unit,obs_status,decimal,Region
0,Brunei Darussalam,1970,"Population, total",BRN,NaN,NaN,NaN,0,ASEAN
1,Brunei Darussalam,1970,"Population, total",BRN,458949.0,NaN,NaN,0,ASEAN
2,Brunei Darussalam,1970,"Population, total",BRN,455370.0,NaN,NaN,0,ASEAN
3,Brunei Darussalam,1970,"Population, total",BRN,451721.0,NaN,NaN,0,ASEAN
4,Brunei Darussalam,1970,"Population, total",BRN,447404.0,NaN,NaN,0,ASEAN


In [64]:
# Rename columns for consistency
df_mobile = df_mobile.rename(columns={
    'COUNTRY': 'country',
    'INDICATOR': 'indicator'
})
df_mobile.head(5)

,0,Zimbabwe,Value of mobile money transactions (during the reference year),28.30352381645416,81.76162552111127,63.41012707881038,54.13707044068592,25.05382726933137,20.36137455708725,14.18105263906246,1.494561669799075
0,1,Bulgaria,Value of mobile money transactions (during the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Cameroon,Value of mobile money transactions (during the...,4.341683,16.448906,29.137805,39.887932,53.4096,NaN,NaN,NaN
2,3,"Latvia, Republic of",Value of mobile money transactions (during the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"Comoros, Union of the",Value of mobile money transactions (during the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Tonga,Value of mobile money transactions (during the...,3.645761,3.635763,5.124455,NaN,NaN,NaN,NaN,NaN


In [7]:
# Convert all 'year' columns to datetime (YYYY only)
for df in [df_main, df_wb, df_agri, df_mobile, df_staple]:
    if 'year' not in df.columns:
        if 'date' in df.columns:
            df.rename(columns={'date': 'year'}, inplace=True)
    df['year'] = pd.to_datetime(df['year'], errors='coerce').dt.year

KeyError: 'year'